In [8]:
import numpy as np
import pandas as pd
from datetime import datetime

# Функция для генерации данных с настройкой для каждой группы
def generate_ab_test_data(n_control, n_experiment, start_date, end_date, control_conversion_rate, experiment_conversion_rate, control_arpu_mean, experiment_arpu_mean):
    # Список дней эксперимента
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')
    
    # Генерация дат посещений
    def generate_session_dates(n_sessions):
        return np.random.choice(date_range, size=n_sessions, replace=True)

    # Вероятностные распределения
    product_views_dist = np.random.poisson(5, size=n_control + n_experiment)  # Поассоново распределение для просмотров
    cart_adds_dist = np.random.poisson(2, size=n_control + n_experiment)  # Поассоново распределение для добавлений в корзину
    session_duration_dist = np.random.exponential(scale=10, size=n_control + n_experiment)  # Экспоненциальное распределение времени сессий
    
    # Распределение для device_type, traffic_source, region
    device_type_dist = np.random.choice(['mobile', 'desktop', 'tablet'], p=[0.7, 0.25, 0.05], size=n_control + n_experiment)
    traffic_source_dist = np.random.choice(['organic', 'paid_ad', 'direct'], p=[0.5, 0.3, 0.2], size=n_control + n_experiment)
    region_dist = np.random.choice(['Estonia', 'Latvia', 'Lithuania'], p=[0.3, 0.4, 0.3], size=n_control + n_experiment)

    # Генерация данных для контрольной группы
    control_data = {
        'user_id': np.arange(1, n_control + 1),
        'group': ['control'] * n_control,
        'session_date': generate_session_dates(n_control),
        'product_views': product_views_dist[:n_control],
        'cart_adds': cart_adds_dist[:n_control],
        'purchase_amount': np.where(np.random.rand(n_control) < control_conversion_rate,  # Вероятность конверсии для контрольной группы
                                    np.random.exponential(scale=control_arpu_mean, size=n_control), 0),  # Экспоненциальное распределение суммы покупок
        'session_duration': session_duration_dist[:n_control],
        'device_type': device_type_dist[:n_control],
        'traffic_source': traffic_source_dist[:n_control],
        'region': region_dist[:n_control]
    }

    # Генерация данных для экспериментальной группы
    experiment_data = {
        'user_id': np.arange(n_control + 1, n_control + n_experiment + 1),
        'group': ['experiment'] * n_experiment,
        'session_date': generate_session_dates(n_experiment),
        'product_views': product_views_dist[n_control:],
        'cart_adds': cart_adds_dist[n_control:],
        'purchase_amount': np.where(np.random.rand(n_experiment) < experiment_conversion_rate,  # Вероятность конверсии для экспериментальной группы
                                    np.random.exponential(scale=experiment_arpu_mean, size=n_experiment), 0),  # Экспоненциальное распределение суммы покупок
        'session_duration': session_duration_dist[n_control:],
        'device_type': device_type_dist[n_control:],
        'traffic_source': traffic_source_dist[n_control:],
        'region': region_dist[n_control:]
    }

    # Преобразуем в DataFrame
    df_control = pd.DataFrame(control_data)
    df_experiment = pd.DataFrame(experiment_data)
    
    # Объединяем контрольную и экспериментальную группы
    df = pd.concat([df_control, df_experiment], ignore_index=True)
    
    # Сохраняем датасет
    #df.to_csv('rimi_ab_test_data_with_exponential_purchase.csv', index=False)

    return df

# Настраиваем параметры для контрольной и экспериментальной групп
n_control = 4361
n_experiment = 4361
start_date = '2024-08-05'
end_date = '2024-08-14'
control_conversion_rate = 0.04  # 5% вероятность конверсии для контрольной группы
experiment_conversion_rate = 0.05  # 7% вероятность конверсии для экспериментальной группы
control_arpu_mean = 50  # Средний ARPU для контрольной группы
experiment_arpu_mean = 53  # Средний ARPU для экспериментальной группы

# Генерация данных
df = generate_ab_test_data(n_control, n_experiment, start_date, end_date, control_conversion_rate, experiment_conversion_rate, control_arpu_mean, experiment_arpu_mean)

# Пример первых строк датасета
print(df.head())


   user_id    group        session_date  product_views  cart_adds  \
0        1  control 2024-08-12 12:00:00              7          1   
1        2  control 2024-08-05 00:00:00              3          1   
2        3  control 2024-08-08 09:00:00              3          1   
3        4  control 2024-08-05 22:00:00              6          3   
4        5  control 2024-08-08 14:00:00              1          4   

   purchase_amount  session_duration device_type traffic_source     region  
0              0.0          5.687580      mobile        paid_ad     Latvia  
1              0.0          5.360450      mobile        paid_ad  Lithuania  
2              0.0          9.915499      mobile        paid_ad     Latvia  
3              0.0          1.924414      mobile         direct     Latvia  
4              0.0         10.854257     desktop        organic     Latvia  


C:\Users\ireev\AppData\Local\Temp\ipykernel_24428\564204609.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq='H')


In [5]:
df.head()

,user_id,group,session_date,product_views,cart_adds,purchase_amount,session_duration,device_type,traffic_source,region
0,1,control,2024-08-11 21:00:00,4,2,0.0,18.954885,mobile,direct,Lithuania
1,2,control,2024-08-14 00:00:00,5,4,0.0,21.493969,mobile,direct,Latvia
2,3,control,2024-08-13 12:00:00,5,3,0.0,7.522270,mobile,organic,Estonia
3,4,control,2024-08-07 04:00:00,2,1,0.0,0.647962,mobile,organic,Estonia
4,5,control,2024-08-08 19:00:00,4,1,0.0,10.834557,desktop,direct,Estonia
